In [ ]:
%matplotlib inline
import kagglehub
imakash3011_customer_personality_analysis_path = kagglehub.dataset_download('imakash3011/customer-personality-analysis')

print('Data source import complete.')


Data source import complete.


# Import Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

/kaggle/input/customer-personality-analysis/marketing_campaign.csv


# Data Importing

In [ ]:
df = pd.read_csv(r"/kaggle/input/customer-personality-analysis/marketing_campaign.csv",sep='\t')
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


# Data Insights

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [ ]:
df.isna().sum()

,0
ID,0
Year_Birth,0
Education,0
Marital_Status,0
Income,24
Kidhome,0
Teenhome,0
Dt_Customer,0
Recency,0
MntWines,0


In [ ]:
df.duplicated().sum()

np.int64(0)

In [ ]:
df.describe()

,ID,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
count,2240.000000,2240.000000,2216.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,...,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.0,2240.0,2240.000000
mean,5592.159821,1968.805804,52247.251354,0.444196,0.506250,49.109375,303.935714,26.302232,166.950000,37.525446,...,5.316518,0.072768,0.074554,0.072768,0.064286,0.013393,0.009375,3.0,11.0,0.149107
std,3246.662198,11.984069,25173.076661,0.538398,0.544538,28.962453,336.597393,39.773434,225.715373,54.628979,...,2.426645,0.259813,0.262728,0.259813,0.245316,0.114976,0.096391,0.0,0.0,0.356274
min,0.000000,1893.000000,1730.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
25%,2828.250000,1959.000000,35303.000000,0.000000,0.000000,24.000000,23.750000,1.000000,16.000000,3.000000,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
50%,5458.500000,1970.000000,51381.500000,0.000000,0.000000,49.000000,173.500000,8.000000,67.000000,12.000000,...,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
75%,8427.750000,1977.000000,68522.000000,1.000000,1.000000,74.000000,504.250000,33.000000,232.000000,50.000000,...,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
max,11191.000000,1996.000000,666666.000000,2.000000,2.000000,99.000000,1493.000000,199.000000,1725.000000,259.000000,...,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.0,11.0,1.000000


In [ ]:
df.shape

(2240, 29)

In [ ]:
columns = df.columns
for col in columns:
    print(f"Unique values in column '{col}':")
    print(df[col].unique())
    print("-" * 30)

Unique values in column 'ID':
[5524 2174 4141 ... 7270 8235 9405]
------------------------------
Unique values in column 'Year_Birth':
[1957 1954 1965 1984 1981 1967 1971 1985 1974 1950 1983 1976 1959 1952
 1987 1946 1980 1949 1982 1979 1951 1969 1986 1989 1963 1970 1973 1943
 1975 1996 1968 1964 1977 1978 1955 1966 1988 1948 1958 1972 1960 1945
 1991 1962 1953 1961 1956 1992 1900 1893 1990 1947 1899 1993 1994 1941
 1944 1995 1940]
------------------------------
Unique values in column 'Education':
['Graduation' 'PhD' 'Master' 'Basic' '2n Cycle']
------------------------------
Unique values in column 'Marital_Status':
['Single' 'Together' 'Married' 'Divorced' 'Widow' 'Alone' 'Absurd' 'YOLO']
------------------------------
Unique values in column 'Income':
[58138. 46344. 71613. ... 56981. 69245. 52869.]
------------------------------
Unique values in column 'Kidhome':
[0 1 2]
------------------------------
Unique values in column 'Teenhome':
[0 1 2]
------------------------------
Unique

# Data cleaning and Preprocessing




## Handle Missing Values

In [ ]:
df['Income'].isna().sum()

np.int64(24)

In [ ]:
fig_before = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=('Box Plot Before Imputation', 'Histogram Before Imputation')
)

box_plot_trace_before = px.box(
    df,
    y='Income',
    points="all"
).data[0]

fig_before.add_trace(box_plot_trace_before, row=1, col=1)

hist_plot_trace_before = px.histogram(
    df,
    x='Income',
    nbins=30
).data[0]

fig_before.add_trace(hist_plot_trace_before, row=1, col=2)

fig_before.update_layout(
    title_text='Income Distribution Before Missing Value Imputation and Transformation',
    title_font_size=20,
    showlegend=False,
    height=500,
    width=1000
)

fig_before.update_yaxes(title_text='Income', row=1, col=1)
fig_before.update_xaxes(title_text='Income', row=1, col=2)
fig_before.update_yaxes(title_text='Number of Persons', row=1, col=2)

fig_before.show()

In [ ]:
df['Income'].fillna(df['Income'].median(), inplace=True)
df['Income'].isna().sum()

/tmp/ipython-input-99-746961944.py:1: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





np.int64(0)

## Handle Skewness With Transformation

In [ ]:
df['Income_transformed'] = np.log1p(df['Income'])

In [ ]:
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=('Box Plot After Transformation', 'Histogram After Transformation')
)
box_plot_trace = px.box(
    df,
    y='Income_transformed',
    points="all"
).data[0]

fig.add_trace(box_plot_trace, row=1, col=1)

hist_plot_trace = px.histogram(
    df,
    x='Income_transformed',
    nbins=30
).data[0]

fig.add_trace(hist_plot_trace, row=1, col=2)

fig.update_layout(
    title_text='Income Distribution After Median Imputation and Log Transformation',
    title_font_size=20,
    showlegend=False,
    height=500,
    width=1000
)

fig.update_yaxes(title_text='Income (Transformed)', row=1, col=1)
fig.update_xaxes(title_text='Income (Transformed)', row=1, col=2)
fig.update_yaxes(title_text='Number of Persons', row=1, col=2)

fig.show()

## Drop Useless Columns

In [ ]:
df.drop(columns=['ID','Income','Z_CostContact','Z_Revenue'], inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year_Birth           2240 non-null   int64  
 1   Education            2240 non-null   object 
 2   Marital_Status       2240 non-null   object 
 3   Kidhome              2240 non-null   int64  
 4   Teenhome             2240 non-null   int64  
 5   Dt_Customer          2240 non-null   object 
 6   Recency              2240 non-null   int64  
 7   MntWines             2240 non-null   int64  
 8   MntFruits            2240 non-null   int64  
 9   MntMeatProducts      2240 non-null   int64  
 10  MntFishProducts      2240 non-null   int64  
 11  MntSweetProducts     2240 non-null   int64  
 12  MntGoldProds         2240 non-null   int64  
 13  NumDealsPurchases    2240 non-null   int64  
 14  NumWebPurchases      2240 non-null   int64  
 15  NumCatalogPurchases  2240 non-null   i

## Change Categorical to Numerical Data

In [ ]:
numeric_col = [col for col in df.columns if df[col].dtype != 'object']
categorical_col = [col for col in df.columns if df[col].dtype == 'object']

In [ ]:
print("Numeric Columns:")
print(numeric_col)
print("\nCategorical Columns:")
print(categorical_col)

Numeric Columns:
['Year_Birth', 'Kidhome', 'Teenhome', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Response', 'Income_transformed']

Categorical Columns:
['Education', 'Marital_Status', 'Dt_Customer']


In [ ]:
df['Education'].value_counts()

,count
Education,
Graduation,1127
PhD,486
Master,370
2n Cycle,203
Basic,54


In [ ]:
df['Marital_Status'].value_counts()

,count
Marital_Status,
Married,864
Together,580
Single,480
Divorced,232
Widow,77
Alone,3
Absurd,2
YOLO,2


In [ ]:
education_map = {
    'Basic': 1,
    '2n Cycle': 2,
    'Graduation': 3,
    'Master': 4,
    'PhD': 5
}

df['EducationLevel'] = df['Education'].map(education_map)

In [ ]:
df = pd.get_dummies(df, columns=['Marital_Status'], drop_first=True)

In [ ]:
df.drop(columns=['Education'], inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Year_Birth               2240 non-null   int64  
 1   Kidhome                  2240 non-null   int64  
 2   Teenhome                 2240 non-null   int64  
 3   Dt_Customer              2240 non-null   object 
 4   Recency                  2240 non-null   int64  
 5   MntWines                 2240 non-null   int64  
 6   MntFruits                2240 non-null   int64  
 7   MntMeatProducts          2240 non-null   int64  
 8   MntFishProducts          2240 non-null   int64  
 9   MntSweetProducts         2240 non-null   int64  
 10  MntGoldProds             2240 non-null   int64  
 11  NumDealsPurchases        2240 non-null   int64  
 12  NumWebPurchases          2240 non-null   int64  
 13  NumCatalogPurchases      2240 non-null   int64  
 14  NumStorePurchases       

In [ ]:
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], errors='coerce')
reference_date = df['Dt_Customer'].dropna().max()
df['Customer_Seniority_Days'] = (reference_date - df['Dt_Customer']).dt.days

if df['Customer_Seniority_Days'].isnull().any():
    median_seniority = df['Customer_Seniority_Days'].median()
    df['Customer_Seniority_Days'].fillna(median_seniority, inplace=True)

df.drop(columns=['Dt_Customer'], inplace=True, errors='ignore')

/tmp/ipython-input-112-609416630.py:7: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





## Data Scaling

In [ ]:
features_for_clustering = df[['Income_transformed', 'EducationLevel',
                              'Recency', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases',
                              'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4',
                              'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2',
                              'Complain', 'Marital_Status_Divorced', 'Marital_Status_Married',
                              'Marital_Status_Single', 'Marital_Status_Together', 'Marital_Status_Widow',
                              'Marital_Status_YOLO', 'Marital_Status_Alone',
                              'Customer_Seniority_Days'
                             ]].copy() # Create a copy to avoid SettingWithCopyWarning

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_for_clustering)

scaled_df = pd.DataFrame(scaled_features, columns=features_for_clustering.columns, index=df.index)
print("\nShape of the data after scaling:", scaled_df.shape)
print("First 5 rows of data after scaling:")
print(scaled_df.head())


Shape of the data after scaling: (2240, 22)
First 5 rows of data after scaling:
   Income_transformed  EducationLevel   Recency  NumDealsPurchases  \
0            0.428874       -0.458383  0.307039           0.349414   
1           -0.021774       -0.458383 -0.383664          -0.168236   
2            0.843209       -0.458383 -0.798086          -0.685887   
3           -1.121817       -0.458383 -0.798086          -0.168236   
4            0.434166        1.533425  1.550305           1.384715   

   NumWebPurchases  NumCatalogPurchases  NumStorePurchases  NumWebVisitsMonth  \
0         1.409304             2.510890          -0.550785           0.693904   
1        -1.110409            -0.568720          -1.166125          -0.130463   
2         1.409304            -0.226541           1.295237          -0.542647   
3        -0.750450            -0.910898          -0.550785           0.281720   
4         0.329427             0.115638           0.064556          -0.130463   

   Accepted

# Data Modeling

In [ ]:
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9]
wcss = []

for num_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50, init='k-means++', random_state=42, n_init='auto')
    kmeans.fit(scaled_df)
    wcss.append(kmeans.inertia_)

    print(f"Inertia for {num_clusters} clusters: {kmeans.inertia_}")

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=range_n_clusters,
    y=wcss,
    mode='lines+markers',
    marker=dict(size=8, color='blue'),
    line=dict(color='blue')
))

fig.update_layout(
    title='The Elbow Method',
    xaxis_title='Number of clusters',
    yaxis_title='WCSS (Within-Cluster Sum of Squares)',
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        tickmode='array',
        tickvals=range_n_clusters
    ),
    yaxis=dict(showgrid=True, gridcolor='lightgray'),
    template='plotly_white'
)

fig.show()

Inertia for 2 clusters: 43777.93293303165
Inertia for 3 clusters: 41925.068274232704
Inertia for 4 clusters: 39126.33593712991
Inertia for 5 clusters: 36867.9804246673
Inertia for 6 clusters: 35224.39382413672
Inertia for 7 clusters: 33972.2011279549
Inertia for 8 clusters: 30565.570752952633
Inertia for 9 clusters: 28306.11852101783


In [ ]:
pca = PCA(n_components=2)
df_pca = pca.fit_transform(scaled_df)

## No of clusters = 3

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
y_kmeans = kmeans.fit_predict(df_pca)

In [ ]:
df_pca = pd.DataFrame(df_pca, columns=['PC1', 'PC2'])
fig = px.scatter(df_pca, x='PC1', y='PC2', color=y_kmeans, color_discrete_sequence='viridis')

fig.update_layout(
    title='K-Means Clustering (PCA)',
    xaxis_title='Principal Component 1',
    yaxis_title='Principal Component 2',
    showlegend=False
)

fig.update_xaxes(showgrid=True, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridcolor='lightgray')

fig.show()

## No of clusters = 4

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42, n_init='auto')
y_kmeans = kmeans.fit_predict(df_pca)

In [ ]:
df_pca = pd.DataFrame(df_pca, columns=['PC1', 'PC2'])
fig = px.scatter(df_pca, x='PC1', y='PC2', color=y_kmeans, color_discrete_sequence='viridis')

fig.update_layout(
    title='K-Means Clustering (PCA)',
    xaxis_title='Principal Component 1',
    yaxis_title='Principal Component 2',
    showlegend=False
)

fig.update_xaxes(showgrid=True, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridcolor='lightgray')

fig.show()

## No of clusters = 7




In [ ]:
kmeans = KMeans(n_clusters=7, random_state=42, n_init='auto')
y_kmeans = kmeans.fit_predict(df_pca)

In [ ]:
df_pca = pd.DataFrame(df_pca, columns=['PC1', 'PC2'])
fig = px.scatter(df_pca, x='PC1', y='PC2', color=y_kmeans, color_discrete_sequence='viridis')

fig.update_layout(
    title='K-Means Clustering (PCA)',
    xaxis_title='Principal Component 1',
    yaxis_title='Principal Component 2',
    showlegend=False
)

fig.update_xaxes(showgrid=True, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridcolor='lightgray')

fig.show()

## No of clusters = 8

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=42, n_init='auto')
y_kmeans = kmeans.fit_predict(df_pca)

In [ ]:
df_pca = pd.DataFrame(df_pca, columns=['PC1', 'PC2'])
fig = px.scatter(df_pca, x='PC1', y='PC2', color=y_kmeans, color_discrete_sequence='viridis')

fig.update_layout(
    title='K-Means Clustering (PCA)',
    xaxis_title='Principal Component 1',
    yaxis_title='Principal Component 2',
    showlegend=False
)

fig.update_xaxes(showgrid=True, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridcolor='lightgray')

fig.show()

## Silhouette Analysis

In [ ]:
print("--- Silhouette Analysis ---")

range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9]
silhouette_scores = []

for num_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50, init='k-means++', random_state=42, n_init='auto')

    y_kmeans = kmeans.fit_predict(df_pca)

    score = silhouette_score(df_pca, y_kmeans)
    silhouette_scores.append(score)

    print(f"Silhouette Score for {num_clusters} clusters: {score:.4f}")
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=range_n_clusters,
    y=silhouette_scores,
    mode='lines+markers',
    marker=dict(size=8, color='purple'),
    line=dict(color='purple')
))
fig.update_layout(
    title='Silhouette Score for Various Numbers of Clusters',
    xaxis_title='Number of clusters',
    yaxis_title='Silhouette Score',
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        tickmode='array', # Ensure all numbers in range_n_clusters are ticks
        tickvals=range_n_clusters
    ),
    yaxis=dict(showgrid=True, gridcolor='lightgray'),
    template='plotly_white' # Use a clean white background
)

fig.show()

print("\n--- Interpretation: ---")
print("Choose the number of clusters that gives the highest Silhouette Score.")
print("A score closer to +1 indicates better defined and separated clusters.")
print("A score near 0 indicates overlapping clusters.")
print("A score closer to -1 indicates data points assigned to the wrong clusters.")

--- Silhouette Analysis ---
Silhouette Score for 2 clusters: 0.4532
Silhouette Score for 3 clusters: 0.4490
Silhouette Score for 4 clusters: 0.4319
Silhouette Score for 5 clusters: 0.3717
Silhouette Score for 6 clusters: 0.3682
Silhouette Score for 7 clusters: 0.3614
Silhouette Score for 8 clusters: 0.3378
Silhouette Score for 9 clusters: 0.3431



--- Interpretation: ---
Choose the number of clusters that gives the highest Silhouette Score.
A score closer to +1 indicates better defined and separated clusters.
A score near 0 indicates overlapping clusters.
A score closer to -1 indicates data points assigned to the wrong clusters.


In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42, n_init='auto')
y_kmeans = kmeans.fit_predict(df_pca)

In [ ]:
df_pca = pd.DataFrame(df_pca, columns=['PC1', 'PC2'])
fig = px.scatter(df_pca, x='PC1', y='PC2', color=y_kmeans, color_discrete_sequence='viridis')

fig.update_layout(
    title='K-Means Clustering (PCA)',
    xaxis_title='Principal Component 1',
    yaxis_title='Principal Component 2',
    showlegend=False
)

fig.update_xaxes(showgrid=True, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridcolor='lightgray')

fig.show()